In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [31]:
import pandas as pd
df = pd.read_csv("reviews_cleaned.csv")


/home/varun/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,2,4,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment,subjectivity,comments_cleaned
0,1419,38924112.0,2015-07-19,11308465.0,Marcela,Having the opportunity of arriving to Alexandr...,0.410104,0.699479,opportunity arriving alexandra house describes...
1,1419,44791978.0,2015-08-29,9580285.0,Marco,We have no enough words to describe how beauty...,0.266667,0.683333,enough word describe beautyful cozy alexandra ...
2,1419,45957133.0,2015-09-07,38394721.0,Andrea,The listing was exceptional and an even better...,0.576111,0.717361,listing exceptional even better experience per...
3,1419,67295154.0,2016-03-28,3515044.0,Shaun,Alexandra's home was amazing and in such a nea...,0.295000,0.563333,alexandra home amazing neat neighbourhood ever...
4,1419,177702208.0,2017-08-03,13987100.0,Kate,Beautiful home. Very comfortable and clean. Pe...,0.502963,0.740000,beautiful home comfortable clean perfect famil...


In [33]:
df["comments_cleaned"] = df["comments_cleaned"].astype(str)

In [34]:
reviews = df["comments_cleaned"]

In [49]:
len(reviews)

609669

In [50]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(reviews)

In [36]:
X

<609669x83635 sparse matrix of type '<class 'numpy.float64'>'
	with 9434192 stored elements in Compressed Sparse Row format>

In [51]:
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [52]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()


Top terms per cluster:


In [69]:
import pickle
pickle.dump(model, open("model.bin","wb"))
pickle.dump(order_centroids, open("order_centroids.bin","wb"))
pickle.dump(terms, open("terms.bin","wb"))

In [70]:
!ls -l *.bin

-rw-r--r-- 1 varun varun 9130099 Apr  8 10:46 model.bin
-rw-r--r-- 1 varun varun 6690963 Apr  8 10:46 order_centroids.bin
-rw-r--r-- 1 varun varun 1471545 Apr  8 10:46 terms.bin


In [90]:
review_cluster = []

In [91]:
for i in reviews.index:    
    Y = vectorizer.transform([reviews[i]])
    prediction = model.predict(Y)   
    review_cluster.append(prediction)

In [92]:
len(review_cluster)

609669

In [100]:
df["clusterid"] = [x[0] for x in review_cluster]

In [102]:
df.to_csv("reviews_cluster.csv", index=False, header=True)

In [101]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment,subjectivity,comments_cleaned,clusterid
0,1419,38924112.0,2015-07-19,11308465.0,Marcela,Having the opportunity of arriving to Alexandr...,0.410104,0.699479,opportunity arriving alexandra house describes...,4
1,1419,44791978.0,2015-08-29,9580285.0,Marco,We have no enough words to describe how beauty...,0.266667,0.683333,enough word describe beautyful cozy alexandra ...,4
2,1419,45957133.0,2015-09-07,38394721.0,Andrea,The listing was exceptional and an even better...,0.576111,0.717361,listing exceptional even better experience per...,2
3,1419,67295154.0,2016-03-28,3515044.0,Shaun,Alexandra's home was amazing and in such a nea...,0.295000,0.563333,alexandra home amazing neat neighbourhood ever...,4
4,1419,177702208.0,2017-08-03,13987100.0,Kate,Beautiful home. Very comfortable and clean. Pe...,0.502963,0.740000,beautiful home comfortable clean perfect famil...,4


In [56]:
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print



Cluster 0:
 stay
 nicole
 david
 week
 safe
 wonderful
 friendly
 place
 fairness
 faire
Cluster 1:
 nan
 zzzs
 fairly
 faire
 fairen
 fairer
 fairerweise
 faireview
 fairey
 fairlie
Cluster 2:
 great
 location
 place
 host
 stay
 clean
 recommend
 apartment
 definitely
 view
Cluster 3:
 great
 place
 location
 stay
 host
 clean
 view
 recommend
 definitely
 value
Cluster 4:
 place
 stay
 clean
 good
 toronto
 host
 apartment
 location
 great
 perfect
Cluster 5:
 nice
 place
 clean
 location
 great
 host
 stay
 really
 good
 room
Cluster 6:
 easy
 check
 great
 location
 place
 clean
 stay
 apartment
 host
 communication
Cluster 7:
 automated
 canceled
 posting
 reservation
 arrival
 day
 host
 fairness
 faire
 fainants
Cluster 8:
 space
 great
 location
 clean
 stay
 host
 comfortable
 recommend
 definitely
 perfect
Cluster 9:
 amazing
 view
 place
 location
 host
 great
 stay
 clean
 definitely
 recommend


In [59]:
print("Prediction")

Y = vectorizer.transform(["excellent place to stay"])
prediction = model.predict(Y)
print(prediction)

Prediction
[4]


In [61]:
Y = vectorizer.transform(["Not funny"])
prediction = model.predict(Y)
print(prediction)

[4]


In [42]:
Y = vectorizer.transform(["Close to the subway station"])
prediction = model.predict(Y)
print(prediction)

[0]
